# Пример дополнения моделей

В данном примере показано, как с помощью программного интерфейса (API) можно дополнять частично определенные модели СКФС. В качестве прикладной области используются модели предприятий, основанные на публичном наборе данных SAP-SAM.

Для запуска примера должны быть заранее обучены генеративные модели с возможностью дополнения для набора данных SAP-SAM. Например, так:

```
python main.py --rules sapsam --data_dir demo_data\sapsam_aug \
               --non_conditional --non_parametric --train_completion
```


In [9]:
import sys
# Для загрузки модулей из основного каталога OrGAN,
# который находится в родительском каталоге
sys.path.insert(0, '..')

In [10]:
import numpy as np

from organ.solver import Solver
from organ.demo import SapSamEMStructureModel
from organ.config import make_config

# Сформируем конфигурацию
config = make_config(rules=SapSamEMStructureModel(),
                     # Папка, в которой находятся данные набора
                     # SAP SAM
                     data_dir='../demo_data/sapsam_aug',
                     # В этом наборе данных нет параметров вершин
                     conditional=False,
                     parametric=False,
                     # Модель должна быть обучена на дополнение
                     completion=True,
                     # Папка, в которой находится модель
                     model_save_dir='../output/models_sapsam_compl',
                     # Идентификатор загружаемой модели
                     # (номер итерации обучения, в результате
                     # которой была получена данная модель)
                     test_iters=4100,
                     mode='test'
                    )

solver = Solver(config)


Max nodes: 22
Node types: 22 22
Edge types: 2 2
CompletionGenerator(
  (noise_encoder): FCBlock(
    (layers): Sequential(
      (0): Linear(in_features=8, out_features=128, bias=True)
      (1): Tanh()
      (2): Linear(in_features=128, out_features=256, bias=True)
      (3): Tanh()
      (4): Linear(in_features=256, out_features=512, bias=True)
      (5): Tanh()
    )
  )
  (global_to_nodes): Linear(in_features=512, out_features=66, bias=True)
  (nodes_to_hidden): Sequential(
    (0): Linear(in_features=5, out_features=10, bias=True)
    (1): ReLU()
  )
  (nodes_encoder): Sequential(
    (0): Linear(in_features=20, out_features=10, bias=True)
    (1): ReLU()
    (2): Linear(in_features=10, out_features=1, bias=True)
  )
  (edges_spec_layer): Linear(in_features=512, out_features=32, bias=True)
  (edge_layers): FCBlock(
    (layers): Sequential(
      (0): Linear(in_features=54, out_features=128, bias=True)
      (1): Tanh()
      (2): Linear(in_features=128, out_features=64, bias=True

## "Простой" интерфейс дополнения

Для использования функции дополнения конфигурации необходимо создать частично определенную конфигурацию. Это делается с помощью указания пар "элементы конфигурации" и соответствующей маски для разных составных частей описания СКФС.

Например, параметр `nodes` представляет собой вектор, содержащий признаки вершин, которые должны присутствовать или отсутствовать в конфигурации, а параметр `nodes_mask` указывает, следует ли генератору "обращать внимание" на соответствующий компонент вектора `nodes` - если значение маски равно `True`, то, по возможности, в генерируемой СКФС наличие соответствующей вершины должно совпадать с наличием ее в `nodes` (если в `nodes` 0, то вершина должна отсутствовать, если нет, то присутствовать), если же значение маски равно `False`, то наличие элемента не играет роли.


In [15]:
nodes = np.zeros(22)
nodes[2] = 2
nodes_mask = np.zeros(22).astype(np.int8)
nodes_mask[2] = True    # Требуем, чтобы вершина 2 была в выходной конфигурции
nodes_mask[8] = True    # Требуем, чтобы вершины 8 не было (поскольку в nodes соответствующий компонент равен 0)

for i in range(22):
    if nodes[i] > 0 and nodes_mask[i] == 1:
        print("Подразделение ", i, ": ", config.rules.node_type_dict[i]["title"], " должно присутствовать")
    elif nodes[i] == 0 and nodes_mask[i] == 1:
        print("Подразделение ", i, ": ", config.rules.node_type_dict[i]["title"], " должно отсутствовать")
orgs = solver.complete(2, nodes=nodes, nodes_mask=nodes_mask)

Подразделение  2 :  Sales  должно присутствовать
Подразделение  8 :  Research & Development  должно отсутствовать
Loading the trained models from step 4100...


In [24]:
print(orgs[0].nodes)
for node in orgs[0].nodes:
    if node > 0:
        print("Подразделение ", node, ": ", config.rules.node_type_dict[node]["title"])

[ 0  1  2  0  4  5  0  7  0  9 10  0  0 13  0  0  0  0  0  0  0 21]
Подразделение  1 :  Managment
Подразделение  2 :  Sales
Подразделение  4 :  Finance
Подразделение  5 :  Logistics
Подразделение  7 :  Purchase
Подразделение  9 :  Human Resources
Подразделение  10 :  Legal
Подразделение  13 :  Analytics
Подразделение  21 :  Warehousing


In [25]:
print(orgs[0].edges)
for i in range(22):
    for j in range(22):
        if orgs[0].edges[i,j] > 0:
            print("Связь (", i, ", ", j, ") от ", 
                  config.rules.node_type_dict[i]["title"], " к ", config.rules.node_type_dict[j]["title"]
                 )

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 0 1 1 0 1 0 1 1 0 0 1 0 0 0 0 0 0 0 1]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0

## Дополнение заведомо валидными структурами

In [32]:
orgs = solver.complete_valid(10, nodes=nodes, nodes_mask=nodes_mask)

Loading the trained models from step 4100...


In [33]:
print(len(orgs))

10


In [34]:
print(orgs[0].nodes)
for node in orgs[0].nodes:
    if node > 0:
        print("Подразделение ", node, ": ", config.rules.node_type_dict[node]["title"])

[ 0  1  2  3  4  5  0  7  0  0 10  0  0 13 14 15  0  0  0  0  0  0]
Подразделение  1 :  Managment
Подразделение  2 :  Sales
Подразделение  3 :  Production
Подразделение  4 :  Finance
Подразделение  5 :  Logistics
Подразделение  7 :  Purchase
Подразделение  10 :  Legal
Подразделение  13 :  Analytics
Подразделение  14 :  Quality Management
Подразделение  15 :  Operations


In [35]:
print(orgs[0].edges)
for i in range(22):
    for j in range(22):
        if orgs[0].edges[i,j] > 0:
            print("Связь (", i, ", ", j, ") от ", 
                  config.rules.node_type_dict[i]["title"], " к ", config.rules.node_type_dict[j]["title"]
                 )

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 1 1 1 1 0 1 0 0 1 0 0 1 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0